In [15]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn 
import os
import time

In [16]:
working_directory = os.path.realpath(os.path.join(os.getcwd(), os.path.dirname('__file__')))[:-7] + 'data\\'

start = time.time()
auctions = pd.read_csv(working_directory + 'auctions.csv.gzip', compression='gzip')
clicks = pd.read_csv(working_directory + 'clicks.csv.gzip', compression='gzip')
events = pd.read_csv(working_directory + 'events.csv.gzip', compression='gzip')
installs = pd.read_csv(working_directory + 'installs.csv.gzip', compression='gzip')
target_competencia = pd.read_csv(working_directory + 'target_competencia.csv')
print('se tardo {0} segundos en cargar dataframes'.format(time.time()-start))

C:\Users\Tomas\Anaconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (12,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


se tardo 96.72134566307068 segundos en cargar dataframes


In [13]:
len(auctions)

19571319